<a href="https://colab.research.google.com/github/Kushagra2000/Neural_machine_translation/blob/main/Machine_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#imports
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)
import pandas as pd
import numpy as np
import re

TensorFlow 1.x selected.
1.15.2


In [2]:
#Upload the deu.txt file and put it in the Data Folder
!ls Data/

deu.txt


In [3]:
#reading lines from thge txt file
with open('Data/deu.txt','r') as f:
  lines=(f.readlines())

In [7]:
#removing the unnecessary data and obtaining sentence pairs 
def process_text(lines):
  ''' 
  Takes a list of tab seperated
  English and German sentences as input 
  and returns list of 
  English and German sentence pairs
  '''
  proc_lines=[]
  for line in lines:
    line=line.strip() #to remove newlines at the end of a sentence
    line=line.split("\t") #splitting by tabs
    line=line[:-1]  #remove contributing information

    line[0]=re.sub(r"([.,?!])",r" \1 ",line[0])   #adding spaces before and after punctuation
    line[0]=re.sub(r"[ ]+",r" ",line[0])  #removing excess spaces
    line[0]=line[0].strip() #removing spaces from the end of string
    line[0]="<SOS> "+line[0]+" <EOS>"

    line[1]=re.sub(r"([.,?!])",r" \1 ",line[1])
    line[1]=re.sub(r"[ ]+",r" ",line[1])
    line[1]=line[1].strip()
    line[1]="<SOS> "+line[1]+" <EOS>"

    proc_lines.append(line) 
  return pd.DataFrame.from_records(data=proc_lines,columns=['eng','ger']) 

In [10]:
df=process_text(lines)
df.head(10)


,eng,ger
0,<SOS> Go . <EOS>,<SOS> Geh . <EOS>
1,<SOS> Hi . <EOS>,<SOS> Hallo ! <EOS>
2,<SOS> Hi . <EOS>,<SOS> Grüß Gott ! <EOS>
3,<SOS> Run ! <EOS>,<SOS> Lauf ! <EOS>
4,<SOS> Run . <EOS>,<SOS> Lauf ! <EOS>
5,<SOS> Wow ! <EOS>,<SOS> Potzdonner ! <EOS>
6,<SOS> Wow ! <EOS>,<SOS> Donnerwetter ! <EOS>
7,<SOS> Fire ! <EOS>,<SOS> Feuer ! <EOS>
8,<SOS> Help ! <EOS>,<SOS> Hilfe ! <EOS>
9,<SOS> Help ! <EOS>,<SOS> Zu Hülf ! <EOS>


In [11]:
class Vocab_builder():
  def __init__(self,lang,series):
    self.lang=lang
    self.data=series
  def tokenize(self,line):
    return line.split(' ')
  def build_vocab(self):
    self.uniq_words=set()
    count=0
    self.num_list=[]
    for index,line in self.data.items():
      self.word_list=tokenize(line)
      for word in self.word_list:
        if(word not in uniq_words):
          uniq_words.add(word)
          num_list.append(count)
          count+=1
      
    vocab_list=list(uniq_words)
    print("Built vocabulary having {} elements".format(len(vocab_list)))
    return zip(vocab_list,num_list)




